In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
import data.processed.load_data_processed_denormed as ldpd
from src.models.CRPS_baseline.CRPS_load import *

2023-05-29 13:04:28.830719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-29 13:04:28.830740: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

In [3]:
# Basics
import numpy as np
import keras.backend as K
import tensorflow as tf

# Norms
from scipy.stats import norm

# Helpful
from tqdm import tqdm

<IPython.core.display.Javascript object>

### 1. Load Raw Data

In [4]:
# Load train data for variables t2m and ws10
train_t2m_ws10_denormed = ldpd.load_data_t2m_ws10_train_proc_denorm()
train_t2m_denormed = train_t2m_ws10_denormed[0]
train_ws10_denormed = train_t2m_ws10_denormed[1]

# Load test data for variables t2m and ws10
test_t2m_ws10_denormed = ldpd.load_data_t2m_ws10_test_proc_denorm()
test_t2m_denormed = test_t2m_ws10_denormed[0]
test_ws10_denormed = test_t2m_ws10_denormed[1]

# Load train and test data for normed t2m:
train_t2m_normed = ldp.load_data_t2m_train_proc_norm()
test_t2m_normed = ldp.load_data_t2m_test_proc_norm()

<IPython.core.display.Javascript object>

###  2. Data Split

In [5]:
# Train-test split into lead_times for t2m
X_train_t2m_denormed, y_train_t2m_denormed = split_var_lead_one(train_t2m_denormed)
X_test_t2m_denormed, y_test_t2m_denormed = split_var_lead_one(test_t2m_denormed)
# Train-test split into lead_times for ws10
X_train_ws10_denormed, y_train_ws10_denormed = split_var_lead_one(train_ws10_denormed)
X_test_ws10_denormed, y_test_ws10_denormed = split_var_lead_one(test_ws10_denormed)

# Train-test split into lead_times for t2m normed
X_train_t2m_normed, y_train_t2m_normed = split_var_lead_one(train_t2m_normed)
X_test_t2m_normed, y_test_t2m_normed = split_var_lead_one(test_t2m_normed)

<IPython.core.display.Javascript object>

Test with z500 denormed, to see whether it also leads to bad results:

In [6]:
train_z500_denormed = ldpd.load_data_z500_train_proc_denorm()
X_train_z500_denormed, y_train_z500_denormed = split_var_lead_one(train_z500_denormed)
train_u10_denormed = ldpd.load_data_u10_train_proc_denorm()
X_train_u10_denormed, y_train_u10_denormed = split_var_lead_one(train_u10_denormed)
train_v10_denormed = ldpd.load_data_v10_train_proc_denorm()
X_train_v10_denormed, y_train_v10_denormed = split_var_lead_one(train_v10_denormed)
train_t850_denormed = ldpd.load_data_t850_train_proc_denorm()
X_train_t850_denormed, y_train_t850_denormed = split_var_lead_one(train_t850_denormed)

<IPython.core.display.Javascript object>

### 3. Train EMOS model

In [7]:
crps_normal(
    mu=X_train_t2m_denormed[0].isel(mean_std=0).values,
    sigma=X_train_t2m_denormed[0].isel(mean_std=1).values,
    y=y_train_t2m_denormed[0].values,
).mean()

0.49815792585237906

<IPython.core.display.Javascript object>

In [8]:
EMOS_model = build_EMOS_network_keras(compile=True, lr=0.1)
EMOS_model.fit(
    [
        X_train_t2m_denormed[0].isel(mean_std=0).values.flatten(),
        X_train_t2m_denormed[0].isel(mean_std=1).values.flatten(),
    ],
    y_train_t2m_denormed[0].values.flatten(),
    epochs=10,
    batch_size=y_train_t2m_denormed[0].values.flatten().shape[0],
)

2023-05-29 13:04:39.901652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-29 13:04:39.901796: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-29 13:04:39.901902: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-29 13:04:39.902005: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file:

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 733.1121
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 6559.7266
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: 732.9309
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 6559.5449
Epoch 5/10
1/1 [==============================] - 1s 998ms/step - loss: 732.7493
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: 6559.3638
Epoch 7/10
1/1 [==============================] - 1s 1s/step - loss: 732.5679
Epoch 8/10
1/1 [==============================] - 1s 1s/step - loss: 6559.1821
Epoch 9/10
1/1 [==============================] - 1s 992ms/step - loss: 732.3863
Epoch 10/10
1/1 [==============================] - 1s 974ms/step - loss: 6559.0005


<IPython.core.display.Javascript object>

In [9]:
EMOS_model.get_weights()

[array([[-1.5755901]], dtype=float32),
 array([0.], dtype=float32),
 array([[1.8109722]], dtype=float32),
 array([0.5681561], dtype=float32)]

<IPython.core.display.Javascript object>

##### Checking that crps_cost_function and crps_normal are equivalent:

In [11]:
crps_cost_function(
    y_test_t2m_denormed[0].values.flatten(),
    np.column_stack(
        (
            X_test_t2m_denormed[0].isel(mean_std=0).values.flatten(),
            X_test_t2m_denormed[0].isel(mean_std=1).values.flatten(),
        )
    ),
    theano=True,
)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5084816>

<IPython.core.display.Javascript object>

In [12]:
crps_normal(
    mu=X_test_t2m_denormed[0].isel(mean_std=0).values.flatten(),
    sigma=X_test_t2m_denormed[0].isel(mean_std=1).values.flatten(),
    y=y_test_t2m_denormed[0].values.flatten(),
).mean()

0.5084816639136494

<IPython.core.display.Javascript object>

In [38]:
EMOS_model = build_EMOS_network_keras(
    compile=True, lr=0.01, loss=crps_cost_function_trunc, optimizer="Adam"
)
EMOS_model.fit(
    [
        X_train_t2m_denormed[0].isel(mean_std=0).values.flatten(),
        X_train_t2m_denormed[0].isel(mean_std=0).values.flatten(),
    ],
    y_train_t2m_denormed[0].values.flatten(),
    epochs=5,
    batch_size=4096,
)

EMOS_preds = EMOS_model.predict(
    [
        X_test_t2m_denormed[0][:, 0, 0, 0].values.flatten(),
        X_test_t2m_denormed[0][:, 0, 0, 1].values.flatten(),
    ]
)


EMOS_crps = crps_trunc(
    mu=EMOS_preds[:, 0],
    sigma=EMOS_preds[:, 1],
    y=y_test_t2m_denormed[0][:, 0, 0].values.flatten(),
)

EMOS_preds_train = EMOS_model.predict(
    [
        X_train_t2m_denormed[0][:, 0, 0, 0].values.flatten(),
        X_train_t2m_denormed[0][:, 0, 0, 1].values.flatten(),
    ]
)

EMOS_crps_train = crps_trunc(
    mu=EMOS_preds_train[:, 0],
    sigma=EMOS_preds_train[:, 1],
    y=y_train_t2m_denormed[0][:, 0, 0].values.flatten(),
)

print(EMOS_crps.mean())

Epoch 1/5
5443/5443 [==============================] - 14s 2ms/step - loss: 1.3556
Epoch 2/5
5443/5443 [==============================] - 12s 2ms/step - loss: 0.4755
Epoch 3/5
5443/5443 [==============================] - 13s 2ms/step - loss: 0.4649
Epoch 4/5
5443/5443 [==============================] - 12s 2ms/step - loss: 0.4643
Epoch 5/5
5443/5443 [==============================] - 13s 2ms/step - loss: 0.4745
0.29842925


<IPython.core.display.Javascript object>

In [32]:
EMOS_crps_train.mean()

0.3002926

<IPython.core.display.Javascript object>

In [33]:
EMOS_model.get_weights()

[array([[1.0012704]], dtype=float32),
 array([-0.3395547], dtype=float32),
 array([[-0.00136477]], dtype=float32),
 array([-0.42161223], dtype=float32)]

<IPython.core.display.Javascript object>

In [34]:
EMOS_preds[:10]

array([[279.56125   ,  -0.42320257],
       [274.85236   ,  -0.42331877],
       [274.09467   ,  -0.4234879 ],
       [277.13617   ,  -0.4233884 ],
       [277.06296   ,  -0.42338008],
       [280.60995   ,  -0.42284018],
       [278.09607   ,  -0.42313257],
       [279.27286   ,  -0.4231175 ],
       [280.07468   ,  -0.42352486],
       [279.84708   ,  -0.42279232]], dtype=float32)

<IPython.core.display.Javascript object>

In [35]:
(y_test_t2m_denormed[0][:, 0, 0].values.flatten())[:10]

array([279.10947, 274.3845 , 274.365  , 276.21277, 277.36887, 279.84338,
       277.7407 , 280.20386, 280.0431 , 279.68265], dtype=float32)

<IPython.core.display.Javascript object>

Things seem to work for ws10 data

In [37]:
EMOS_model = build_EMOS_network_keras(
    compile=True, lr=0.001, loss=crps_cost_function_trunc, optimizer="Adam"
)
EMOS_model.fit(
    [
        X_train_ws10_denormed[0].isel(mean_std=0).values.flatten(),
        X_train_ws10_denormed[0].isel(mean_std=1).values.flatten(),
    ],
    y_train_ws10_denormed[0].values.flatten(),
    epochs=5,
    batch_size=8192,
)

EMOS_preds = EMOS_model.predict(
    [
        X_test_ws10_denormed[0].isel(mean_std=0).values.flatten(),
        X_test_ws10_denormed[0].isel(mean_std=1).values.flatten(),
    ],
    verbose=1,
)
### I think

EMOS_crps = crps_trunc(
    mu=EMOS_preds[:, 0],
    sigma=EMOS_preds[:, 1],
    y=y_test_ws10_denormed[0].values.flatten(),
)

# EMOS_preds_train = EMOS_model.predict(
#     [
#         X_train_ws10_denormed[0].isel(mean_std=0).values.flatten(),
#         X_train_ws10_denormed[0].isel(mean_std=1).values.flatten(),
#     ],
#     verbose=1,
# )

# EMOS_crps_train = crps_trunc(
#     mu=EMOS_preds_train[:, 0],
#     sigma=EMOS_preds_train[:, 1],
#     y=y_train_ws10_denormed[0].values.flatten(),
# )

print(EMOS_crps.mean())

Epoch 1/5
2722/2722 [==============================] - 9s 3ms/step - loss: 0.7460
Epoch 2/5
2722/2722 [==============================] - 8s 3ms/step - loss: 0.6382
Epoch 3/5
2722/2722 [==============================] - 8s 3ms/step - loss: 0.6381
Epoch 4/5
2722/2722 [==============================] - 8s 3ms/step - loss: 0.6381
Epoch 5/5
174038/174038 [==============================] - 109s 626us/step
0.4714637


<IPython.core.display.Javascript object>

In [21]:
EMOS_model.get_weights()

[array([[1.0570519]], dtype=float32),
 array([-0.00596516], dtype=float32),
 array([[-0.69789577]], dtype=float32),
 array([-0.36045372], dtype=float32)]

<IPython.core.display.Javascript object>

In [28]:
EMOS_crps = crps_trunc(
    mu=EMOS_preds[:, 0],
    sigma=EMOS_preds[:, 1],
    y=y_test_ws10_denormed[0].values.flatten(),
)

<IPython.core.display.Javascript object>

In [29]:
EMOS_crps.mean()

0.46506056

<IPython.core.display.Javascript object>

In [24]:
y_test_ws10_denormed[0].values.flatten()

array([21.866837, 21.810654, 21.780798, ...,  8.717077,  9.446845,
        9.298131], dtype=float32)

<IPython.core.display.Javascript object>

#### 3.1 Loading baseline to compare

In [57]:
ws10_crps_baseline = crps_load_lead_lat_lon("ws10")
t2m_crps_baseline = crps_load_lead_lat_lon("t2m")

<IPython.core.display.Javascript object>

In [59]:
print(f"Wind baseline: {ws10_crps_baseline[0].mean()}")
print(f"T2m baseline: {t2m_crps_baseline[0].mean()}")

Wind baseline: 0.7447009086608887
T2m baseline: 0.9354666921537256


<IPython.core.display.Javascript object>

### 3. Train EMOS_global model for each lead_time and variable

In [87]:
def EMOS_global_train(
    var_num,
    lead_time,
    batch_size=4096,
    epochs=10,
    lr=0.001,
    validation_split=0.2,
    optimizer="Adam",
):
    """
    Train all global EMOS models for a specific variable and lead_time

    Args: 
        var_num (integer): number between 0 - 5 for each of the variables["u10", "v10", "t2m", "t850", "z500", "ws10"]
        lead_time (integer): number between 0 - 30 for each lead_time
    """

    # Adjust lead_time for 1-based indexing
    lead_time = lead_time + 1

    # Define the names of the variables
    var_names = ["u10", "v10", "t2m", "t850", "z500", "ws10"]

    # Load the training data
    train_var_denormed = ldpd.load_data_all_train_proc_denorm()[var_num]

    # Split the data into features and target
    X_train_var_denormed = train_var_denormed[
        list(train_var_denormed.data_vars.keys())[0]
    ].isel(lead_time=lead_time)
    y_train_var_denormed = train_var_denormed[
        list(train_var_denormed.data_vars.keys())[1]
    ].isel(lead_time=lead_time)

    # Define the cost function depending on the variable number
    if var_num in [5]:
        crps = crps_cost_function_trunc
    else:
        crps = crps_cost_function

    # Build and compile the model
    EMOS_glob = build_EMOS_network_keras(
        compile=True, lr=lr, loss=crps, optimizer=optimizer
    )

    # Define the filename for the model checkpoint
    model_filename = (
        "/home/dchen/BA_CH_EN/models/EMOS_global_models/denormed/EMOS_glob_"
        + var_names[var_num]
        + "_lead_time_"
        + str(lead_time - 1)
        + "_denormed.h5"
    )

    # Define callbacks for early stopping and model checkpointing
    early_stopping = EarlyStopping(monitor="val_loss", patience=3)
    model_checkpoint = ModelCheckpoint(
        model_filename, monitor="val_loss", mode="min", save_best_only=True
    )

    # Fit the model to the training data
    EMOS_glob.fit(
        [
            X_train_var_denormed.isel(mean_std=0).values.flatten(),
            X_train_var_denormed.isel(mean_std=1).values.flatten(),
        ],
        y_train_var_denormed.values.flatten(),
        batch_size=batch_size,
        epochs=epochs,
        validation_split=validation_split,
        callbacks=[early_stopping, model_checkpoint],
    )

    # Load the best model and return
    best_model = tf.keras.models.load_model(
        model_filename, 
        custom_objects={
            "crps_cost_function": crps_cost_function,
            "crps_cost_function_trunc": crps_cost_function_trunc
        }
    )

    return best_model


<IPython.core.display.Javascript object>

In [88]:
def EMOS_global_predict_evaluate(EMOS_glob, var_num, lead_time):
    """
    Use the given EMOS_global model to predict and evaluate on a test dataset.
    Args:
        EMOS_glob: the EMOS global model
        var_num (integer): number between 0 - 5 for each of the variables["u10", "v10", "t2m", "t850", "z500", "ws10"]
        lead_time (integer): number between 0 - 30 for each lead_time
    """

    # Increment lead_time by one to use in indexing
    lead_time = lead_time + 1

    # Define the variable names
    var_names = ["u10", "v10", "t2m", "t850", "z500", "ws10"]

    # Load test data
    test_var_denormed = ldpd.load_data_all_test_proc_denorm()[var_num]

    # Split test data into features and targets
    X_test_var_denormed = test_var_denormed[
        list(test_var_denormed.data_vars.keys())[0]
    ].isel(lead_time=lead_time)

    y_test_var_denormed = test_var_denormed[
        list(test_var_denormed.data_vars.keys())[1]
    ].isel(lead_time=lead_time)

    # Choose the CRPS cost function based on variable number
    if var_num in [5]:
        crps = crps_trunc
    else:
        crps = crps_normal

    # Use the EMOS model to make predictions
    EMOS_glob_preds = EMOS_glob.predict(
        [
            X_test_var_denormed.isel(mean_std=0).values.flatten(),
            X_test_var_denormed.isel(mean_std=1).values.flatten(),
        ],
        verbose=1,
    )

    # Evaluate the model predictions using the chosen CRPS function
    EMOS_glob_crps = crps(
        mu=EMOS_glob_preds[:, 0],
        sigma=EMOS_glob_preds[:, 1],
        y=y_test_var_denormed.values.flatten(),
    )
    
    # Reshape the CRPS scores and compute the mean along the first axis
    EMOS_glob_crps = EMOS_glob_crps.reshape(y_test_var_denormed.shape).mean(axis=0)

    # Save the average CRPS score over all days for 120 x 130 grid
    model_filename = f"/Data/Delong_BA_Data/scores/EMOS_global_scores/EMOS_global_{var_names[var_num]}_lead_{lead_time}_scores.npy"
    np.save(model_filename, EMOS_glob_crps)

<IPython.core.display.Javascript object>

In [89]:
def run_all(
    var_num, lead_time, batch_size=4096, epochs=10, lr=0.001, validation_split=0.2
):
    EMOS_glob = EMOS_global_train(
        var_num,
        lead_time,
        batch_size=batch_size,
        epochs=epochs,
        lr=lr,
        validation_split=validation_split,
    )
    EMOS_global_predict_evaluate(EMOS_glob, var_num, lead_time)

<IPython.core.display.Javascript object>

In [90]:
EMOS_glob_t2m_0_scores, EMOS_glob_t2m_0_preds = run_all(5, 0, epochs=10)

Epoch 1/10
4354/4354 [==============================] - 13s 3ms/step - loss: 0.7619 - val_loss: 0.6286
Epoch 2/10
4354/4354 [==============================] - 11s 3ms/step - loss: 0.6409 - val_loss: 0.6287
Epoch 3/10
4354/4354 [==============================] - 12s 3ms/step - loss: 0.6408 - val_loss: 0.6277
Epoch 4/10
4354/4354 [==============================] - 11s 3ms/step - loss: 0.6408 - val_loss: 0.6285
Epoch 5/10
4354/4354 [==============================] - 11s 3ms/step - loss: 0.6408 - val_loss: 0.6278
Epoch 6/10
4354/4354 [==============================] - 11s 3ms/step - loss: 0.6408 - val_loss: 0.6274
Epoch 7/10
4354/4354 [==============================] - 11s 3ms/step - loss: 0.6408 - val_loss: 0.6275
Epoch 8/10
4354/4354 [==============================] - 11s 3ms/step - loss: 0.6408 - val_loss: 0.6276
Epoch 9/10
174038/174038 [==============================] - 106s 610us/step


<IPython.core.display.Javascript object>

In [91]:
EMOS_glob_t2m_0_scores.mean()

0.4697708

<IPython.core.display.Javascript object>

In [69]:
EMOS_glob_t2m_0_scores.mean()

0.46876928

<IPython.core.display.Javascript object>

In [71]:
EMOS_glob_t2m_0_preds.shape

(357, 120, 130, 2)

<IPython.core.display.Javascript object>

In [72]:
EMOS_glob_t2m_0_scores.shape

(120, 130)

<IPython.core.display.Javascript object>

In [73]:
EMOS_glob_t2m_0_scores

array([[0.6543617 , 0.6332479 , 0.6167404 , ..., 0.31890804, 0.30976433,
        0.30422524],
       [0.6369018 , 0.6295203 , 0.6334581 , ..., 0.3348354 , 0.31547937,
        0.3043394 ],
       [0.57872915, 0.5657393 , 0.5672672 , ..., 0.28843513, 0.28858635,
        0.28958994],
       ...,
       [0.4628801 , 0.4697256 , 0.48701125, ..., 0.60685045, 0.62224823,
        0.6275593 ],
       [0.549715  , 0.6716716 , 0.4389424 , ..., 0.63595545, 0.6325761 ,
        0.6329455 ],
       [0.6199186 , 0.50849265, 0.96981126, ..., 0.62486637, 0.62932765,
        0.6119128 ]], dtype=float32)

<IPython.core.display.Javascript object>

In [60]:
120 * 130

15600

<IPython.core.display.Javascript object>